# ReACT Agent

## SETUP

In [20]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
MISTRALAI_API_KEY = os.getenv("MISTRALAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

## SIMPLE CALCULATOR TOOL

In [21]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.mistralai import MistralAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

### DEFINE TOOLS

In [22]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a * b

In [23]:
# initiate tools
tool_multiply = FunctionTool.from_defaults(fn=multiply)
tool_add = FunctionTool.from_defaults(fn=add)
tool_subtract = FunctionTool.from_defaults(fn=subtract)

### OPENAI LLM

In [24]:
llm_openai = OpenAI(api_key=OPENAI_API_KEY, model="gpt-4")

agent_openai = ReActAgent.from_tools(
    [tool_multiply, tool_add, tool_subtract], llm=llm_openai, verbose=True
)

In [6]:
# get response from agent
response_openai = agent_openai.chat("What is 20+(2*4)? Calculate step by step.")

Thought: The user wants to calculate the expression 20+(2*4) step by step. The first step is to calculate the multiplication part of the expression, which is 2*4. I will use the 'multiply' tool for this.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
Thought: The result of the multiplication 2*4 is 8. Now, I need to add this result to 20. I will use the 'add' tool for this.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
Thought: I can answer without using any more tools. The result of the addition 20+8 is 28, which is the final result of the expression 20+(2*4).
Answer: The result of the expression 20+(2*4) is 28.


In [7]:
print(response_openai)

The result of the expression 20+(2*4) is 28.


### ANTHROPIC LLM

In [26]:
llm_anthropic = Anthropic(api_key=ANTHROPIC_API_KEY, model="claude-3-sonnet-20240229")

In [ ]:

agent_anthropic = ReActAgent.from_tools(
    [tool_add, tool_multiply, tool_subtract], llm=llm_anthropic, verbose=True
)

response_anthropic = agent_anthropic.chat("What is 20+(2*4)? Calculate step by step.")

print(response_anthropic)

### MISTRAL LLM

In [27]:
llm_mistral = MistralAI(api_key=MISTRALAI_API_KEY, model="mistral-large-latest")
agent_mistral = ReActAgent.from_tools(
    [tool_add, tool_multiply, tool_subtract], llm=llm_mistral, verbose=True
)

In [ ]:

response_mistral = agent_mistral.chat("What is 20+(2*4)? Calculate step by step.")
print(response_mistral)

### CHECK PROMPT

In [ ]:
prompt_dict_openai = agent_openai.get_prompts()
for k, v in prompt_dict_openai.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t

: 

## SETUP

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
MISTRALAI_API_KEY = os.getenv("MISTRALAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

## RAG QUERY ENGINE TOOLS

In [10]:
from llama_index.core import (
    # SimpleDirectoryReader,
    # VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.legacy import VectorStoreIndex
from llama_index.legacy.readers import SimpleDirectoryReader

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [11]:
# NOTE: index means a node, a document can be split up into multiple nodes/chunks
try:
    storage_context = StorageContext.from_defaults(persist_dir="../data/storage/lyft")
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(persist_dir="../data/storage/uber")
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [12]:
# # Download data
# !mkdir -p '../data/llamaindex/data/10k'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O '../data/llamaindex/data/10k/uber_2021.pdf'
# !wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O '../data/llamaindex/data/10k/lyft_2021.pdf'

### LOAD DATA

In [13]:
# from llama_index.core import readers

if not index_loaded:
    lyft_docs = SimpleDirectoryReader(
        input_files=["../data/llamaindex/data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["../data/llamaindex/data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="../data/storage/lyft")
    uber_index.storage_context.persist(persist_dir="../data/storage/uber")

### CREATE INDEX

In [14]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

## DEFINE TOOLS

In [17]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

### OPENAI

In [29]:
agent_openai = ReActAgent.from_tools(query_engine_tools, llm=llm_openai, verbose=True)

In [31]:
response_openai = agent_openai.chat("Compare the revenue growth of Uber and Lyft in 2021.")

Thought: To compare the revenue growth of Uber and Lyft in 2021, I need to use the uber_10k and lyft_10k tools to get the financial information for each company.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: Uber's revenue growth in 2021 was 57%.
Thought: Now that I have the revenue growth for Uber, I need to use the lyft_10k tool to get the revenue growth for Lyft in 2021.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: In 2021, Uber's revenue growth was 57%, while Lyft's revenue increased by 36%. Therefore, Uber had a higher revenue growth compared to Lyft in 2021.


In [32]:
print(response_openai)

In 2021, Uber's revenue growth was 57%, while Lyft's revenue increased by 36%. Therefore, Uber had a higher revenue growth compared to Lyft in 2021.


In [34]:
agent_anthropic = ReActAgent.from_tools(
    tools=query_engine_tools, llm=llm_anthropic, verbose=True
)

response_anthropic = agent_anthropic.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)

print(response_anthropic)

Thought: The current language of the user is: English. I need to use tools to get information about Uber and Lyft's revenue growth in 2021 to answer this question.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: Uber's revenue growth in 2021 was 57%.
Thought: I now have information about Uber's revenue growth in 2021, but still need Lyft's revenue growth to compare the two companies.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Thought: I now have the revenue growth information for both Uber and Lyft in 2021. I can answer the question without using any more tools.
Answer: In 2021, Uber's revenue grew at a faster rate of 57% compared to Lyft's revenue growth of 36%.
In 2021, Uber's revenue grew at a faster rate of 57% compared to Lyft's revenue growth of 36%.


In [35]:
agent_mistral = ReActAgent.from_tools(
    tools=query_engine_tools, llm=llm_mistral, verbose=True
)

response_mistral = agent_mistral.chat(
    "Compare the revenue growth of Uber and Lyft in 2021."
)
print(response_mistral)

Thought: The current language of the user is: English. I need to use tools to help me answer the question. I will first use the 'lyft_10k' tool to get information about Lyft's revenue in 2021, then use the 'uber_10k' tool to get information about Uber's revenue in 2021.
Action: lyft_10k
Action Input: {'input': 'What was the revenue of Lyft in 2021?'}
Observation: Lyft's revenue in 2021 was $3,208,323,000.
Thought: The current language of the user is: English. I have the information about Lyft's revenue in 2021. Now, I need to use the 'uber_10k' tool to get information about Uber's revenue in 2021.
Action: uber_10k
Action Input: {'input': 'What was the revenue of Uber in 2021?'}
Observation: $17,455
Thought: The current language of the user is: English. I have the information about Uber's and Lyft's revenue in 2021. Now, I can calculate the revenue growth of Uber and Lyft in 2021.

Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Uber's